In [2]:
from datetime import datetime, timedelta
import uuid
from ortools.sat.python import cp_model
import pandas as pd
import pickle
import csv
from IPython.display import clear_output

In [159]:
class Employee:
    def __init__(self, first_name: str, last_name: str, role: str = 'Uncategorized'):
        assert len(first_name) > 0, 'First name cannot be empty'
        assert len(last_name) > 0, 'Last name cannot be empty'
        assert len(role) > 0, 'Role cannot be empty'

        self.first_name = first_name
        self.last_name = last_name
        self.name = first_name + " " + last_name
        self.role = role
        self._id = uuid.uuid4()
        self._created_at = datetime.now()
        self._updated_at = datetime.now()
        self.all_tasks = []

    @property
    def full_name(self) -> str:
        return self.name

    @property
    def shifts(self) -> list:
        shifts = []
        for task in self.all_tasks:
            if isinstance(task, Shift):
                shifts.append(task)
        return shifts

    @property
    def tasks(self) -> list:
        tasks = []
        for task in self.all_tasks:
            if not isinstance(task, Shift) and isinstance(task, Task):
                tasks.append(task)
        return tasks

    def __repr__(self) -> str:
        return f"Employee('{self.name}', '{self.role}', {self.all_tasks})"

    def add_task(self, task):
        # Check if task is already in the list
        if task in self.all_tasks:
            raise Exception('Task is already in the list')
        # Check if task start_time & end_time is not overlapping with any other tasks #TODO: Currently disable
        # for t in self.all_tasks:
        #     if (task.start_time >= t.start_time and task.start_time < t.end_time) or (task.end_time > t.start_time and task.end_time <= t.end_time):
        #         raise Exception('Task overlaps with another task')

        self.all_tasks.append(task)
        self._updated_at = datetime.now()
    
    def remove_task(self, task):
        # Check if task is in the list
        if task not in self.all_tasks:
            raise Exception('Task is not in the list')
        self.all_tasks.remove(task)
        self._updated_at = datetime.now()

    def reset_tasks(self):
        self.all_tasks = []
        self._updated_at = datetime.now()

    #TODO: fix task, all_tasks, and shifts
    def is_available(self, task):
        # Check if task start_time & end_time is not overlapping with any other tasks
        for t in self.tasks:
            if (task.start_time >= t.start_time and task.start_time < t.end_time) or (task.end_time > t.start_time and task.end_time <= t.end_time):
                return False
        return True

    @classmethod
    def from_csv(cls, file_name: str) -> list:
        employees = []
        with open(file_name, 'r') as f:
            reader = csv.reader(f)
            header = next(reader) # save the header for indexing
            for row in reader:
                employees.append(cls(row[header.index('first_name')], row[header.index('last_name')], row[header.index('role')]))
            
        return employees


class Task:
    def __init__(self, name: str, description: str, start_time: datetime, duration: timedelta):
        
        self.name = name
        self.description = description
        assert duration >= timedelta(minutes=0), "duration must be greater than or equal to 0"
        self.start_time = start_time
        self.duration = duration
        self.end_time = start_time + duration
        self._id = uuid.uuid4()
        self._created_at = datetime.now()
        self._updated_at = datetime.now()

    def __repr__(self) -> str:
        return self.name

    # Check if the task is overlapping with another task
    def overlap(self, other) -> bool:
        return self.start_time < other.end_time and other.start_time < self.end_time
    
    # Check if the task is overlapping with a list of tasks
    def overlap_list(self, task_list) -> bool:
        for task in task_list:
            if self.overlap(task):
                return True
        return False
    

class Shift(Task):
    def __init__(self, name: str, description: str, duration: timedelta, start_time: datetime, shift_type: str, min_employees: int, max_employees: int):
        super().__init__(name, description, start_time, duration)
        self.shift_type = str.lower(shift_type)
        assert min_employees <= max_employees, "min_employees must be less than or equal to max_employees"
        assert min_employees >= 0, "min_employees must be greater than or equal to 0"
        assert max_employees >= 0, "max_employees must be greater than or equal to 0"
        self.min_employees = min_employees
        self.max_employees = max_employees
        self.employees = []
        # self.date = start_time.date()

    @property
    def date(self):
        return self.start_time.day

    @property
    def type(self):
        return self.shift_type

    def add_employee(self, employee):
        # Check if employee is already in the list
        if employee in self.employees:
            raise Exception('Employee is already in the list')
        self.employees.append(employee)
        self._updated_at = datetime.now()

    def remove_employee(self, employee):
        # Check if employee is in the list
        if employee not in self.employees:
            raise Exception('Employee is not in the list')
        self.employees.remove(employee)
        self._updated_at = datetime.now()

    def reset_employees(self):
        self.employees = []
        self._updated_at = datetime.now()


# Solution printer.
class ShiftSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shift_vars: dict, shifts: list[Shift], employees: list[Employee], penalties, start_time: datetime, end_time: datetime):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__shift_vars = shift_vars
        self.__shifts = shifts
        self.__employees = employees
        self.__shift_types = set([s.type for s in shifts])
        self.__employee_roles = set([e.role for e in employees])
        self.__solution_count = 0
        self.__start_time = start_time
        self.__end_time = end_time

    @property
    def dates(self):
        return pd.date_range(self.__start_time, self.__end_time, freq='D')

    def on_solution_callback(self):

        # Create a dataframe with the dates as the index and the shift types as the columns
        shift_schedule = pd.DataFrame(index=self.dates, columns=[shift_type for shift_type in self.__shift_types])
        shift_by_type = {}
        # Optimized version
        for shift in self.__shifts:
            if shift.shift_type in shift_by_type:
                shift_by_type[shift.shift_type].append(shift)
            else:
                shift_by_type[shift.shift_type] = [shift]

        # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'Unassigned'
        for date in self.dates:
            for shift_type in shift_by_type:
                for shift in shift_by_type[shift_type]:
                    if shift.start_time.date() == date:
                        shift_schedule.loc[date, shift_type] = [employee.first_name for employee in self.__employees if self.Value(self.__shift_vars[(shift, employee)]) == 1]

        # Fill nan values with '' (empty string)
        shift_schedule = shift_schedule.fillna('')
    
        self.__solution_count += 1

        
        schedule_workload = pd.DataFrame(index=[employee.name for employee in self.__employees], columns=[shift_type for shift_type in self.__shift_types])
            
        for employee in self.__employees:
            for shift_type in self.__shift_types:
                schedule_workload.loc[employee.name, shift_type] = len([shift for shift in shift_by_type[shift_type] if self.Value(self.__shift_vars[(shift, employee)]) == 1])
        
        clear_output(wait=True)
        print('Solution %i' % self.__solution_count)
        print('  Objective value = %i' % self.ObjectiveValue())
        display(shift_schedule)
        display(schedule_workload)

    def solution_count(self):
        return self.__solution_count


class Schedule:

    def __init__(self, name: str, start_time: datetime, end_time: datetime):
        assert start_time < end_time, 'Start time must be before end time'
        self.name = name
        self.start_time = start_time
        self.end_time = end_time
        self.employees = []
        self.shifts = []

        self._holidays = self.get_weekends(start_time, end_time)

        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()

        self.__model = cp_model.CpModel()
        self.__shift_vars = {}
        
        
    @property
    def solution_printer(self):
        return ShiftSolutionPrinter(self.__shift_vars, self.shifts, self.employees, self.__penalties, self.start_time, self.end_time)
    
    @property
    def penalty(self):
        return self.__penalties
    
    @property
    def dates(self) -> list[datetime]:
        return [self.start_time + timedelta(days=x) for x in range(self.duration.days + 1)]

    @property
    def duration(self) -> timedelta:
        return self.end_time - self.start_time

    @property
    def holidays(self) -> list[datetime]:
        return self._holidays

    @property
    def roles(self) -> set:
        return(set([employee.role for employee in self.employees]))

    @property
    def shift_types(self) -> set:
        return set([shift.shift_type for shift in self.shifts])


    def add_holiday(self, date: datetime) -> None:
        # Check if date is in dates 
        if date not in self.dates:
            raise ValueError('Date is not in schedule')
        # Check if date is already a holiday
        if date in self._holidays:
            raise ValueError('Date is already a holiday')
        self._holidays.append(date)
        self.__updated_at = datetime.now()

    def remove_holiday(self, date: datetime) -> None:
        # Check if date is in holidays
        if date in self._holidays:
            self._holidays.remove(date)
            self.__updated_at = datetime.now()
        else:
            raise ValueError('Date is not a holiday')

    @staticmethod
    def get_weekends(start_time: datetime, end_time: datetime) -> list[datetime]:
        return [start_time + timedelta(days=x) for x in range((end_time - start_time).days + 1) if (start_time + timedelta(days=x)).weekday() in [5, 6]]

    def __repr__(self) -> str:
        return f'{self.name} From {self.start_time.isoformat()} to {self.end_time.isoformat()}'

    def __str__(self) -> str:
        return f'{self.name} From {self.start_time.isoformat()} to {self.end_time.isoformat()}'

    def reset(self) -> None:
        self.employees = []
        self.shifts = []
 
    def add_employee(self, employee) -> object:
        self.employees.append(employee)
        self.__updated_at = datetime.now()
        return employee

    def add_shift(self, shift) -> object:
        self.shifts.append(shift)
        self.__updated_at = datetime.now()
        return shift

    def remove_employee(self, employee) -> None:
        self.employees.remove(employee)
        self.__updated_at = datetime.now()

    def remove_shift(self, shift) -> None:
        self.shifts.remove(shift)
        self.__updated_at = datetime.now()

    def assign_shift(self, shift, employee) -> None:
        shift.add_employee(employee)
        employee.add_task(shift)
        self.__updated_at = datetime.now()

    def show(self, format = 'text', group_by = 'shift type') -> None:        
        
        if format == 'text':
            self.__display_text()
        elif format == 'table':
            self.__display_table(group_by=group_by)
        else:
            raise Exception('Invalid format')

    def __display_text(self) -> None:
        print('Schedule Name: {}'.format(self.name))
        print('Start Time: {}'.format(self.start_time))
        print('End Time: {}'.format(self.end_time))
        print('Duration: {}'.format(self.duration))
        print('Employees: {}'.format(self.employees))
        print('Shifts: {}'.format(self.shifts))
        print('Created At: {}'.format(self.__created_at))
        print('Updated At: {}'.format(self.__updated_at))

    def __display_table(self, group_by = 'shift type') -> pd.DataFrame:
        dates = [date.date() for date in pd.date_range(self.start_time, self.end_time, freq='D')]
        shifts = self.shifts
        shift_types = self.shift_types
        employees = self.employees

        if group_by == 'shift':
            # Create a dataframe with the dates as the index and the shifts as the columns
            shift_schedule = pd.DataFrame(index=dates, columns=[shift.name for shift in shifts])

            # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'None'
            # If a shift is assigned to multiple employees, fill with list of employees
            # If there're no shifts on a given day, fill with "-"
            for date in dates:
                for shift in shifts:
                    if shift.start_time.date() == date:
                        if len(shift.employees) == 0:
                            shift_schedule.loc[date, shift.name] = 'None'
                        elif len(shift.employees) == 1:
                            shift_schedule.loc[date, shift.name] = shift.employees[0].first_name
                        else:
                            shift_schedule.loc[date, shift.name] = [employee.first_name for employee in shift.employees]
                    else:
                        shift_schedule.loc[date, shift.name] = '-'
        elif group_by == 'shift type':
            # Create a dataframe with the dates as the index and the shift types as the columns
            shift_schedule = pd.DataFrame(index=dates, columns=[shift_type for shift_type in shift_types])
            shift_by_type = {}
            # Optimized version
            for shift in shifts:
                if shift.shift_type in shift_by_type:
                    shift_by_type[shift.shift_type].append(shift)
                else:
                    shift_by_type[shift.shift_type] = [shift]

            # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'Unassigned'
            for date in dates:
                for shift_type in shift_by_type:
                    for shift in shift_by_type[shift_type]:
                        if shift.start_time.date() == date:
                            if len(shift.employees) == 0:
                                shift_schedule.loc[date, shift_type] = 'Unassigned'
                            elif len(shift.employees) == 1:
                                shift_schedule.loc[date, shift_type] = shift.employees[0].first_name
                            elif len(shift.employees) > 1:
                                shift_schedule.loc[date, shift_type] = [employee.first_name for employee in shift.employees]
            
            # Fill nan values with '' (empty string)
            shift_schedule = shift_schedule.fillna(' ')

        elif group_by == 'workload':
            # Create a dataframe with the employees as the index and the shift types as columns
            shift_schedule = pd.DataFrame(index=[employee.name for employee in employees], columns=[shift_type for shift_type in shift_types])
            
            for employee in self.employees:
                for shift_type in shift_types:
                    shift_schedule.loc[employee.name, shift_type] = len([shift for shift in employee.shifts if shift.shift_type == shift_type])
      
        else:
            raise ValueError(f'Invalid value for group_by: {group_by}')
        
        
        # Display the dataframe
        display(shift_schedule)

        return shift_schedule

    # Solve the schedule with CP-SAT
    def solve(self):
        # self.__model = self.__model

        # Create the variables (shift, employee)
        self.__shift_vars = {}
        for shift in self.shifts:
            for employee in self.employees:
                self.__shift_vars[(shift, employee)] = self.__model.NewBoolVar('shift_{}_employee_{}'.format(shift.name, employee.name))


        # ===================== Constraints ===================== #

        # Each shift must be assigned to employees more than or equal to min_employees, and less than or equal to max_employees
        for shift in self.shifts:
            self.__model.Add(sum(self.__shift_vars[(shift, employee)] for employee in self.employees) >= shift.min_employees)
            self.__model.Add(sum(self.__shift_vars[(shift, employee)] for employee in self.employees) <= shift.max_employees)

        # If the shift is assigned to employees, fixed the shift assigned to the employees
        fixed_shifts = [] # List of tuples (shift, employee)
        for shift in self.shifts:
            for employee in shift.employees:
                fixed_shifts.append((shift, employee))
        for shift, employee in fixed_shifts:
            self.__model.Add(self.__shift_vars[(shift, employee)] == 1)

        # The shift should only be assigned to employees who are available during the shift (compare with employee's tasks) except ems, observe
        exclude_shift_types = ['ems', 'observe']
        except_date = [20, 21, 22, 23, 24]
        for shift in self.shifts:
            for employee in self.employees:
                if not employee.is_available(shift) and (shift.shift_type not in exclude_shift_types) and (shift.start_time.date().day not in except_date):
                    self.__model.Add(self.__shift_vars[(shift, employee)] == 0) # False


        # Logical matrix fot shift type on the same day
        for date in self.dates:
            num_employee_morning = len([employee for employee in self.employees if employee.is_available(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 4)))])
            num_employee_afternoon = len([employee for employee in self.employees if employee.is_available(Task(name='', description='', start_time= date + timedelta(hours = 12), duration=timedelta(hours = 4)))])
            num_employee_allday = len([employee for employee in self.employees if employee.is_available(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 8)))])
            num_shift_morning = len([shift for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 4)))])
            num_shift_afternoon = len([shift for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 12), duration=timedelta(hours = 4)))])
            num_shift_all_day = len([shift for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 8)))])
            
            # Strict constraint
            if (num_employee_morning >= num_shift_morning) and (num_employee_afternoon >= num_shift_afternoon) and (num_employee_allday >= num_shift_all_day):

                print(f'Enough employee for {date} morning shift ({num_employee_morning} >= {num_shift_morning}), afternoon shift ({num_employee_afternoon} >= {num_shift_afternoon}), all day shift ({num_employee_allday} >= {num_shift_all_day})')

                shift_type_array = [
                    ['service 1', 'service 2', 'service 1+', 'service 2+'],
                    ['service 1', 'ems', 'observe'],    
                    ['service 2', 'ems', 'observe'],
                    ['morning conference', 'service 1', 'service 1+'],
                    ['morning conference', 'observe', 'ems'], 
                    ['avd', 'service 1', 'service 2'],
                ]

                for shift_type_group in shift_type_array:
                    for employee in self.employees:
                        for shift1 in self.shifts:
                            for shift2 in self.shifts:
                                if (shift1 != shift2) and (shift1.shift_type in shift_type_group) and (shift2.shift_type in shift_type_group) and (shift1.start_time.date() == shift2.start_time.date() and shift1.start_time.date().day < 16 and date.day == shift1.start_time.date().day):
                                    self.__model.Add((self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)]) <= 1)
                                    # print(f'Add constraint for {shift1} and {shift2} for {employee.first_name} on {shift1.start_time.date()}')


                # for day >= 16 only
                shift_type_array = [
                    ['service 1', 'service 2', 'service 1+', 'service 2+'],
                    ['service 1', 'ems', 'observe'],    
                    ['service 2', 'ems', 'observe'],
                    ['avd', 'service 1', 'service 2', 'service 1+', 'service 2+'],
                    ['avd', 'morning conference']
                ]

                for shift_type_group in shift_type_array:
                    for employee in self.employees:
                        for shift1 in self.shifts:
                            for shift2 in self.shifts:
                                if (shift1 != shift2) and (shift1.shift_type in shift_type_group) and (shift2.shift_type in shift_type_group) and (shift1.start_time.date() == shift2.start_time.date() and shift1.start_time.date().day >= 16 and date.day == shift1.start_time.date().day):
                                    # self.__model.Add((self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)]) <= 1)

                                    # Exclude date 20 - 24
                                    # if shift1.start_time.date().day not in [20, 21, 22, 23, 24]:
                                        self.__model.AddAtMostOne([self.__shift_vars[(shift1, employee)], self.__shift_vars[(shift2, employee)]])

                                    

            # Soft constraint
            else:
                print(f'Not enough employees for shift on {date.date()} morning :({num_employee_morning} employees, {num_shift_morning} shifts), afternoon :({num_employee_afternoon} employees, {num_shift_afternoon} shifts), all day :({num_employee_allday} employees, {num_shift_all_day} shifts)')

                shift_type_array = [
                    ['service 1', 'service 2', 'service 1+', 'service 2+'],
                    ['service 1', 'ems', 'observe'],    
                    ['service 2', 'ems', 'observe'],
                    ['morning conference', 'service 1', 'service 1+'],
                    ['morning conference', 'observe', 'ems'], 
                    ['avd', 'service 1', 'service 2']
                ]

                for shift_type_group in shift_type_array:
                    for employee in self.employees:
                        for shift1 in self.shifts:
                            for shift2 in self.shifts:
                                if (shift1 != shift2) and (shift1.shift_type in shift_type_group) and (shift2.shift_type in shift_type_group) and (shift1.start_time.date() == shift2.start_time.date() and shift1.start_time.date().day < 16 and date.day == shift1.start_time.date().day):
                                    if shift1.start_time.date().day not in [15]:
                                        self.__model.Add((self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)]) <= 1)
                                    # print(f'Add constraint for {shift1} and {shift2} for {employee.first_name} on {shift1.start_time.date()}')


                # for day >= 16 only
                shift_type_array = [
                    ['service 1', 'service 2', 'service 1+', 'service 2+'],
                    # ['service 1', 'service 2'],
                    # ['service 1', 'service 1+', 'service 2+'],
                    # ['service 2', 'service 1+', 'service 2+'],
                    ['service 1', 'ems', 'observe'],    
                    ['service 2', 'ems', 'observe'],
                    ['morning conference', 'service 1', 'service 1+'],
                    ['avd', 'service 1', 'service 2', 'service 1+', 'service 2+'],
                    # ['avd', 'service 1', 'service 2'],
                    ['avd', 'morning conference']
                ]

                for shift_type_group in shift_type_array:
                    for employee in self.employees:
                        for shift1 in self.shifts:
                            for shift2 in self.shifts:
                                if (shift1 != shift2) and (shift1.shift_type in shift_type_group) and (shift2.shift_type in shift_type_group) and (shift1.start_time.date() == shift2.start_time.date() and shift1.start_time.date().day >= 16 and date.day == shift1.start_time.date().day):
                                    # self.__model.Add((self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)]) <= 1)
                                    if shift1.start_time.date().day not in [20, 21, 22, 23, 24]:
                                        self.__model.AddAtMostOne([self.__shift_vars[(shift1, employee)], self.__shift_vars[(shift2, employee)]])


        #TODO: REOPEN THIS        
        # Employee can't work more than 2 shift types in a day, make it 1 if available employee is more than shift in a day
        for employee in self.employees:
            for date in self.dates:
                num_employee_morning = len([employee for employee in self.employees if employee.is_available(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 4)))])
                num_employee_afternoon = len([employee for employee in self.employees if employee.is_available(Task(name='', description='', start_time= date + timedelta(hours = 12), duration=timedelta(hours = 4)))])
                num_shift_morning = len([shift for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 4)))])
                num_shift_afternoon = len([shift for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 12), duration=timedelta(hours = 4)))])

                if num_employee_morning >= num_shift_morning:
                    self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 4)))]) <= 1)
                else:
                    self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 8), duration=timedelta(hours = 4)))]) <= 2)
                
                if num_employee_afternoon >= num_shift_afternoon:
                    self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 12), duration=timedelta(hours = 4)))]) <= 1)
                else:
                    self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.start_time.date().day == date.day and shift.overlap(Task(name='', description='', start_time= date + timedelta(hours = 12), duration=timedelta(hours = 4)))]) <= 2)

                # all day can't work more than 3 shifts
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.start_time.date().day == date.day]) <= 2)

    
        # # # TODO: REOPEN THIS
        # Sum service 1/1+/2/2+ shifts per scedule for each employee ≥ certain number # TODO: fix this
        shift_type1 = ['service 1', 'service 1+', 'service 2', 'service 2+']
        for employee in self.employees:
            if employee.role == 'assisted instructor':
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type1]) >= 7)
            elif employee.role == 'instructor':
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type1]) >= 9)
            elif employee.role == 'assisted executive':
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type1]) >= 5)
            elif employee.role == 'executive':
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type1]) >= 3)

        # # #TODO: REOPEN THIS
        # Each employee can only work total service 1, 2 shifts per schedule ≤ certain number
        shift_type = ['service 1', 'service 2']
        for employee in self.employees:
            self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type]) <= 4)
        # # #TODO: REOPEN THIS
        # Each employee can only work morning conference shifts per schedule ≤ certain number
        shift_type = ['morning conference']
        for employee in self.employees:
            self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type]) >= 1)
            self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type in shift_type]) <= 2)      
        # # #TODO: REOPEN THIS
        # Distance between morning conference > 2 days for each employee
        for employee in self.employees:
            for shift1 in self.shifts:
                for shift2 in self.shifts:
                    if (shift1 != shift2) and (shift1.shift_type == 'morning conference') and (shift2.shift_type == 'morning conference') and (shift1.start_time.date() != shift2.start_time.date()) and (abs((shift1.start_time.date() - shift2.start_time.date()).days) <= 2):
                        self.__model.Add((self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)]) <= 1).OnlyEnforceIf(self.__shift_vars[(shift1, employee)])

        # # Can't work AVD on holiday after date > 16 more than 1 time / schedule
        for employee in self.employees:
            for shift in self.shifts:
                for date in self.holidays:
                    if shift.start_time.date().day > 16 and shift.shift_type == 'avd' and shift.start_time.date().day == date.day:
                        self.__model.Add(self.__shift_vars[(shift, employee)] <= 1)

        # Equalize workload for each employee for avd
        for employee in self.employees:
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd') and (shift.start_time.day < 16)]) <= 2)
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type == 'avd' and (shift.start_time.day >= 16)]) <= 2)
                    
            
        

        #     if employee.role == 'unassigned':
        #         continue
        #     for date in range(1, 16):
        #         workload_per_staff = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd morning on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd morning') and (shift.date == date)] ) == workload_per_staff)
        #         workload_per_staff2 = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd afternoon on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd afternoon') and (shift.date == date)] ) == workload_per_staff2)
        #         delta = self.__model.NewIntVar(0, 1000, f'Delta of {employee.first_name} for avd morning and avd afternoon on {date}')
        #         self.__model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
        #         obj_int_vars.append(delta)
        #         obj_int_coeffs.append(5)
        #     for date in range(16, len(self.dates) + 1):
        #         workload_per_staff = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd morning on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd morning') and (shift.date == date)] ) == workload_per_staff)
        #         workload_per_staff2 = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd afternoon on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd afternoon') and (shift.date == date)] ) == workload_per_staff2)
        #         delta = self.__model.NewIntVar(0, 1000, f'Delta of {employee.first_name} for avd morning and avd afternoon on {date}')
        #         self.__model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
        #         obj_int_vars.append(delta)
        #         obj_int_coeffs.append(1)

     

        # ===================== Objective ===================== #
        obj_int_vars = []
        obj_int_coeffs = []
        obj_bool_vars = []
        obj_bool_coeffs = []

        # TODO: Re-OPEN THIS
        # # Minimize some shift types for each employee on the same day ([shift_type1, shift_type2], coefficient)
        # # for day < 16
        shift_type_array = [
            (['morning conference', 'service 2'], 2),
            (['morning conference', 'service 2+'], 1),
            (['amd', 'service 1'], 2),
            (['amd', 'service 1+'], 1),
            (['amd', 'service 2'], 2),
            (['amd', 'service 2+'], 1),
            (['amd', 'morning conference'], 1),
            (['amd', 'observe'], 1),
            (['amd', 'ems'], 1),
            (['observe', 'service 1+'], 2),
            (['observe', 'service 2+'], 2),
            (['ems', 'service 1+'], 1),
            (['ems', 'service 2+'], 1),
            (['avd', 'service 1'], 2),
            (['avd', 'service 1+'], 0),
            (['avd', 'service 2'], 2),
            (['avd', 'service 2+'], 0),
            (['avd', 'morning conference'], 1),
            (['avd', 'amd'], 1),
            (['avd', 'observe'], 1),
            (['avd', 'ems'], 1),
        ]

        for i in shift_type_array:
            shift_type_group = i[0]
            coefficient = i[1]
            for employee in self.employees:
                for shift1 in self.shifts:
                    for shift2 in self.shifts:
                        if (shift1 != shift2) and (shift1.start_time.date() == shift2.start_time.date()) and (shift1.shift_type in shift_type_group) and (shift2.shift_type in shift_type_group and shift1.start_time.date().day < 16):
                            obj_bool_vars.append(self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)])
                            obj_bool_coeffs.append(coefficient)
       
        # TODO: Re-OPEN THIS
        # # for day >= 16
        shift_type_array = [
            (['morning conference', 'service 2'], 2),
            (['morning conference', 'service 2+'], 1),
            (['amd', 'service 1'], 2),
            (['amd', 'service 1+'], 1),
            (['amd', 'service 2'], 2),
            (['amd', 'service 2+'], 1),
            (['amd', 'morning conference'], 1),
            (['amd', 'observe'], 1),
            (['amd', 'ems'], 1),
            (['observe', 'service 1+'], 2),
            (['observe', 'service 2+'], 2),
            (['ems', 'service 1+'], 1),
            (['ems', 'service 2+'], 1),
            (['avd', 'service 1+'], 1),
            (['avd', 'service 2+'], 1),
            (['avd', 'morning conference'], 1),
            (['avd', 'amd'], 1),
            (['avd', 'observe'], 1),
            (['avd', 'ems'], 1),
        ]

        for i in shift_type_array:
            shift_type_group = i[0]
            coefficient = i[1]
            for employee in self.employees:
                for shift1 in self.shifts:
                    for shift2 in self.shifts:
                        if (shift1 != shift2) and (shift1.start_time.date() == shift2.start_time.date()) and (shift1.shift_type in shift_type_group) and (shift2.shift_type in shift_type_group and shift1.start_time.date().day < 16):
                            obj_bool_vars.append(self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)])
                            obj_bool_coeffs.append(coefficient)


        # # If employee role is 'unassigned', try to minimize the number of shifts that are assigned to the employee
        # for employee in self.employees:
        #     if employee.role == 'unassigned':
        #         for shift in self.shifts:
        #             if shift.shift_type in ['service 1', 'service 2', 'service 1+', 'service 2+']:
        #                 obj_bool_vars.append(self.__shift_vars[(shift, employee)])
        #                 obj_bool_coeffs.append(100000)
        #             else:
        #                 obj_bool_vars.append(self.__shift_vars[(shift, employee)])
        #                 obj_bool_coeffs.append(1000)
        
        # # Each employee can only be assigned to one shift at a time (regarding the start and end time of the shift) 
        # # shift_types = ['service 1', 'service 2', 'service 1+', 'service 2+', 'morning conference']
        # for employee in self.employees:
        #     for shift1 in self.shifts:
        #         for shift2 in self.shifts:
        #             if (shift1 != shift2) and shift1.overlap(shift2):
        #                 obj_bool_vars.append(self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)])
        #                 obj_bool_coeffs.append(100)


        # # for shift in self.shifts:
        # #     priority_point = 0
        # #     if (shift.shift_type == 'observe'):
        # #         priority_point = 100
        # #     elif (shift.shift_type == 'ems'):
        # #         priority_point = 80
        # #     elif (shift.shift_type == 'amd'):
        # #         priority_point = 80
        # #     elif (shift.shift_type == ['avd']) and (shift.date >= 16):
        # #         priority_point = 120
        # #     elif (shift.shift_type == ['avd']) and (shift.date < 16):
        # #         priority_point = 40
                
        # #     obj_int_vars.append(sum(self.__shift_vars[(shift, employee)] for employee in self.employees))
        # #     obj_int_coeffs.append(priority_point * -1)

        # # shift_type1 = ['service 1', 'service 2', 'ems', 'amd', 'observe', 'avd']
        # # for employee in self.employees:
        # #     for shift1 in self.shifts:
        # #         for shift2 in self.shifts:
        # #             if (shift1 != shift2) and (shift1.start_time.date() == shift2.start_time.date()) and (shift1.shift_type in shift_type1) and (shift2.shift_type in shift_type1):
        # #                 obj_int_vars.append(self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)])
        # #                 obj_int_coeffs.append(100)


        # # Minimize some shift types on the same day
        # shift_type1 = ['service 1', 'service 1+', 'service 2', 'service 2+']
        # for employee in self.employees:
        #     if employee.role == 'unassigned':
        #         continue
        #     for shift1 in self.shifts:
        #         for shift2 in self.shifts:
        #             if (shift1 != shift2) and (shift1.start_time.date() == shift2.start_time.date()) and (shift1.shift_type in shift_type1) and (shift2.shift_type in shift_type1):
        #                 obj_int_vars.append(self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)])
        #                 obj_int_coeffs.append(1)
                        
        # shift_type1 = ['morning conference', 'service 2', 'service 2+']
        # for employee in self.employees:
        #     for shift1 in self.shifts:
        #         for shift2 in self.shifts:
        #             if (shift1 != shift2) and (shift1.start_time.date() == shift2.start_time.date()) and (shift1.shift_type in shift_type1) and (shift2.shift_type in shift_type1):
        #                 obj_int_vars.append(self.__shift_vars[(shift1, employee)] + self.__shift_vars[(shift2, employee)])
        #                 obj_int_coeffs.append(1)

                
        # TODO: Re-OPEN THIS
        # Equalize the workload of each employee
        for employee in self.employees:
            if employee.role == 'unassigned':
                continue
            workload_per_staff = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name}')
            self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts] ) == workload_per_staff)
            for employee2 in self.employees:
                if employee2 == employee:
                    continue
                workload_per_staff2 = self.__model.NewIntVar(0, 1000, f'Workload of {employee2.first_name}')
                self.__model.Add(sum([self.__shift_vars[(shift, employee2)] for shift in self.shifts] ) == workload_per_staff2)
                delta = self.__model.NewIntVar(0, 1000, f'Delta of {employee.first_name} and {employee2.first_name}')
                self.__model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
                obj_int_vars.append(delta)
                obj_int_coeffs.append(1)
        # TODO: Re-OPEN THIS
        # Equalize the workload of each employee for each shift type
        for employee in self.employees:
            if employee.role == 'unassigned':
                continue
            for shift_type in self.shift_types:
                workload_per_staff = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for {shift_type}')
                self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type == shift_type] ) == workload_per_staff)
                for employee2 in self.employees:
                    if employee2 == employee:
                        continue
                    workload_per_staff2 = self.__model.NewIntVar(0, 1000, f'Workload of {employee2.first_name} for {shift_type}')
                    self.__model.Add(sum([self.__shift_vars[(shift, employee2)] for shift in self.shifts if shift.shift_type == shift_type] ) == workload_per_staff2)
                    delta = self.__model.NewIntVar(0, 1000, f'Delta of {employee.first_name} and {employee2.first_name} for {shift_type}')
                    self.__model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
                    obj_int_vars.append(delta)
                    obj_int_coeffs.append(1)

        # TODO: Re-OPEN THIS
        # Equalize avd morning and avd afternoon on date 1-15 & 16+ for each employee
        # for employee in self.employees:
        #     if employee.role == 'unassigned':
        #         continue
        #     for date in range(1, 16):
        #         workload_per_staff = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd morning on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd morning') and (shift.date == date)] ) == workload_per_staff)
        #         workload_per_staff2 = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd afternoon on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd afternoon') and (shift.date == date)] ) == workload_per_staff2)
        #         delta = self.__model.NewIntVar(0, 1000, f'Delta of {employee.first_name} for avd morning and avd afternoon on {date}')
        #         self.__model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
        #         obj_int_vars.append(delta)
        #         obj_int_coeffs.append(5)
        #     for date in range(16, len(self.dates) + 1):
        #         workload_per_staff = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd morning on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd morning') and (shift.date == date)] ) == workload_per_staff)
        #         workload_per_staff2 = self.__model.NewIntVar(0, 1000, f'Workload of {employee.first_name} for avd afternoon on {date}')
        #         self.__model.Add(sum([self.__shift_vars[(shift, employee)] for shift in self.shifts if (shift.shift_type == 'avd afternoon') and (shift.date == date)] ) == workload_per_staff2)
        #         delta = self.__model.NewIntVar(0, 1000, f'Delta of {employee.first_name} for avd morning and avd afternoon on {date}')
        #         self.__model.AddAbsEquality(delta, workload_per_staff - workload_per_staff2)
        #         obj_int_vars.append(delta)
        #         obj_int_coeffs.append(1)


        # TODO: Re-OPEN THIS
        # Consecutive shifts (shift type, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost)
        # TODO: Make this recieve a list of tuples from the user
        consecutive_shifts = [
            ('avd',       1, 2, 1, 2, 2, 0), 
            # ('ems',       1, 2, 1, 7, 7, 0),
            # ('observe',   1, 2, 1, 7, 7, 0),
        ]

        for ct in consecutive_shifts:
            shift_type, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost = ct
            for employee in self.employees:
                if employee.role == 'unassigned':
                    continue
                works = [self.__shift_vars[(shift, employee)] for shift in self.shifts if shift.shift_type == shift_type]
                variables, coeffs = Schedule.__add_soft_sequence_constraint(
                    self.__model, works, hard_min, soft_min, min_cost, soft_max, hard_max,
                    max_cost,
                    f'consecutive_shifts(staff {employee.first_name}, {shift_type})')
                obj_bool_vars.extend(variables)
                obj_bool_coeffs.extend(coeffs)


        obj_bool_penalties = sum([coeff * variables for coeff, variables in zip(obj_bool_coeffs, obj_bool_vars)])
        obj_int_penalties = sum([coeff * variables for coeff, variables in zip(obj_int_coeffs, obj_int_vars)])
        self.__penalties = obj_bool_penalties + obj_int_penalties
        self.__model.Minimize(self.__penalties)


        # ===================== Solution ===================== #

        # Solve the model
        solver = cp_model.CpSolver() #TODO: Add parameters to the solver
        status = solver.Solve(self.__model, self.solution_printer)

        # Print the solution
        if (status == cp_model.OPTIMAL) or (status == cp_model.FEASIBLE):
            print("Solution found.")
            # for shift in self.shifts:
            #     for employee in self.employees:
            #         if solver.Value(shifts[(shift, employee)]) == 1:
            #             print('Employee {} is assigned to shift {}'.format(employee.name, shift.name))

        else:
            print('No solution found.')
            return None

        # Update the schedule with the solution
        for shift in self.shifts:
            for employee in self.employees:
                if solver.Value(self.__shift_vars[(shift, employee)]) == 1 and employee not in shift.employees:
                    shift.add_employee(employee)
                    employee.add_task(shift)

        self.__updated_at = datetime.now()

        return self

    # Add Constraint to the model
    def save(self, file_name):
        with open(file_name, 'wb') as f:
            pickle.dump(self, f)

    def to_csv(self, path):
        self.__display_table(group_by="shift type").to_csv(path + '/schedule.csv')
        self.__display_table(group_by="workload").to_csv(path + '/workload.csv')

    @staticmethod
    def load(file_name):
        with open(file_name, 'rb') as f:
            return pickle.load(f)

    @staticmethod
    def __negated_bounded_span(works, start, length):
        """Filters an isolated sub-sequence of variables assined to True.
    Extract the span of Boolean variables [start, start + length), negate them,
    and if there is variables to the left/right of this span, surround the span by
    them in non negated form.
    Args:
        works: a list of variables to extract the span from.
        start: the start to the span.
        length: the length of the span.
    Returns:
        a list of variables which conjunction will be false if the sub-list is
        assigned to True, and correctly bounded by variables assigned to False,
        or by the start or end of works.
    """
        sequence = []
        # Left border (start of works, or works[start - 1])
        if start > 0:
            sequence.append(works[start - 1])
        for i in range(length):
            sequence.append(works[start + i].Not())
        # Right border (end of works or works[start + length])
        if start + length < len(works):
            sequence.append(works[start + length])
        return sequence

    @staticmethod
    def __add_soft_sequence_constraint(model, works, hard_min, soft_min, min_cost,
                                    soft_max, hard_max, max_cost, prefix):
    
        cost_literals = []
        cost_coefficients = []

        # Forbid sequences that are too short.
        for length in range(1, hard_min):
            for start in range(len(works) - length + 1):
                model.AddBoolOr(Schedule.__negated_bounded_span(works, start, length))

        # Penalize sequences that are below the soft limit.
        if min_cost > 0:
            for length in range(hard_min, soft_min):
                for start in range(len(works) - length + 1):
                    span = Schedule.__negated_bounded_span(works, start, length)
                    name = ': under_span(start=%i, length=%i)' % (start, length)
                    lit = model.NewBoolVar(prefix + name)
                    span.append(lit)
                    model.AddBoolOr(span)
                    cost_literals.append(lit)
                    # We filter exactly the sequence with a short length.
                    # The penalty is proportional to the delta with soft_min.
                    cost_coefficients.append(min_cost * (soft_min - length))

        # Penalize sequences that are above the soft limit.
        if max_cost > 0:
            for length in range(soft_max + 1, hard_max + 1):
                for start in range(len(works) - length + 1):
                    span = Schedule.__negated_bounded_span(works, start, length)
                    name = ': over_span(start=%i, length=%i)' % (start, length)
                    lit = model.NewBoolVar(prefix + name)
                    span.append(lit)
                    model.AddBoolOr(span)
                    cost_literals.append(lit)
                    # Cost paid is max_cost * excess length.
                    cost_coefficients.append(max_cost * (length - soft_max))

        # Just forbid any sequence of true variables with length hard_max + 1
        for start in range(len(works) - hard_max):
            model.AddBoolOr(
                [works[i].Not() for i in range(start, start + hard_max + 1)])
        return cost_literals, cost_coefficients

    @staticmethod
    def __add_soft_sum_constraint(model, works, hard_min, soft_min, min_cost,
                                soft_max, hard_max, max_cost, prefix):
        
        cost_variables = []
        cost_coefficients = []
        sum_var = model.NewIntVar(hard_min, hard_max, '')
        # This adds the hard constraints on the sum.
        model.Add(sum_var == sum(works))
        if soft_min > hard_min and min_cost > 0:
            delta = model.NewIntVar(-len(works), len(works), '')
            model.Add(delta == soft_min - sum_var)
            # TODO(user): Compare efficiency with only excess >= soft_min - sum_var.
            excess = model.NewIntVar(0, 7, prefix + ': under_sum')
            model.AddMaxEquality(excess, [delta, 0])
            cost_variables.append(excess)
            cost_coefficients.append(min_cost)

        # Penalize sums above the soft_max target.
        if soft_max < hard_max and max_cost > 0:
            delta = model.NewIntVar(-7, 7, '')
            model.Add(delta == sum_var - soft_max)
            excess = model.NewIntVar(0, 7, prefix + ': over_sum')
            model.AddMaxEquality(excess, [delta, 0])
            cost_variables.append(excess)
            cost_coefficients.append(max_cost)

        return cost_variables, cost_coefficients


In [145]:
s = Shift(name='a', start_time=datetime(2019, 1, 1, 8, 0), duration= timedelta(hours=8), shift_type='a', description='', min_employees=1, max_employees=1)

s.start_time.day

1

In [162]:
# create a schedule for March 2023
schedule = Schedule('March 2023', datetime(2023, 3, 1), datetime(2023, 3, 31))

# add employees
employees = Employee.from_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/inputs/staffs.csv')
for employee in employees:
    schedule.add_employee(employee)

schedule.remove_employee([employee for employee in schedule.employees if employee.first_name == 'วชิระ'][0])

# add shifts (Service 1, service 2, service 1+, service 2+, morning conference, EMS, observe, พอป การบิน (AMD), แพทย์บิน(AVD))
for d in schedule.dates:
    if d.weekday() in [0, 1, 2, 3, 4]:
        # Service 1
        schedule.add_shift(Shift(name=f'Service 1 on {d.date()}',
                                description=f'Service at Emergency area from 8.00 - 12.00',
                                duration=timedelta(hours=4), 
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='service 1',
                                min_employees=1,
                                max_employees=1))
        # Service 1+
        schedule.add_shift(Shift(name=f'Service 1+ on {d.date()}',
                                description=f'Service at Urgency area from 8.00 - 12.00',
                                duration=timedelta(hours=4), 
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='service 1+',
                                min_employees=1,
                                max_employees=1))
        # Service 2
        schedule.add_shift(Shift(name=f'Service 2 on {d.date()}',
                                description=f'Service at Emergency area from 12.00 - 16.00',
                                duration=timedelta(hours=4),
                                start_time=datetime(d.year, d.month, d.day, 12, 0),
                                shift_type='service 2',
                                min_employees=1,
                                max_employees=1))
        # Service 2+
        schedule.add_shift(Shift(name=f'Service 2+ on {d.date()}',
                                description=f'Service at Urgency area from 12.00 - 16.00',
                                duration=timedelta(hours=4),
                                start_time=datetime(d.year, d.month, d.day, 12, 0),
                                shift_type='service 2+',
                                min_employees=1,
                                max_employees=1))
        # Morning conference
        schedule.add_shift(Shift(name=f'Morning conference on {d.date()}',
                                description=f'Morning conference from 8.00 - 10.00',
                                duration=timedelta(hours=2),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='Morning conference',
                                min_employees=1,
                                max_employees=1))
        # EMS
        schedule.add_shift(Shift(name=f'EMS on {d.date()}',
                                description=f'Standby for EMS from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='EMS',
                                min_employees=1,
                                max_employees=1))
        # Observe
        schedule.add_shift(Shift(name=f'Observe on {d.date()}',
                                description=f'Observe from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='observe',
                                min_employees=1,
                                max_employees=1))                   


    # แพทย์บิน (AVD) วันที่ 16 ขึ้นไป
    schedule.add_shift(Shift(name=f'AVD on {d.date()}',
                            description=f'แพทย์บิน (Aviation doctor) from 8.00 - 16.00',
                            duration=timedelta(hours=8),
                            start_time=datetime(d.year, d.month, d.day, 8, 0),
                            shift_type='AVD',
                            min_employees=1,
                            max_employees=1))
    
    
    if d.day >= 16:
        # พอป การบิน (AMD)
        schedule.add_shift(Shift(name=f'AND on {d.date()}',
                                description=f'พอป. การบิน (Air medical director) from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='AMD',
                                min_employees=1,
                                max_employees=1))


schedule.add_holiday(datetime(2023, 3, 6))

# remove shifts
for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 6).day and shift.shift_type == 'service 1']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 6).day and shift.shift_type == 'service 1+']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 6).day and shift.shift_type == 'service 2']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 6).day and shift.shift_type == 'service 2+']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 6).day and shift.shift_type == 'morning conference']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 6).day and shift.shift_type in ['morning conference', 'observe', 'ems', 'observe', 'ems']]:
    schedule.remove_shift(s)


# Remove morning conference on 2023-03-09, 2023-03-10, 2023-03-13, 2023-03-14, 2023-03-15
for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 9).day and shift.shift_type == 'morning conference']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 10).day and shift.shift_type == 'morning conference']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 13).day and shift.shift_type == 'morning conference']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 14).day and shift.shift_type == 'morning conference']:
    schedule.remove_shift(s)

for s in [shift for shift in schedule.shifts if shift.date == datetime(2023, 3, 15).day and shift.shift_type == 'morning conference']:
    schedule.remove_shift(s)

# Assign fix shift to ems and observe shifts
for date in pd.date_range(start='2023-03-01', end='2023-03-03'):
    for shift in schedule.shifts:
        if shift.type == 'ems' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[0])
        if shift.type == 'observe' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[6])
for date in pd.date_range(start='2023-03-07', end='2023-03-10'):
    for shift in schedule.shifts:
        if shift.type == 'ems' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[5])
        if shift.type == 'observe' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[2])
for date in pd.date_range(start='2023-03-13', end='2023-03-17'):
    for shift in schedule.shifts:
        if shift.type == 'ems' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[10])
        if shift.type == 'observe' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[4])     
for date in pd.date_range(start='2023-03-20', end='2023-03-24'):
    for shift in schedule.shifts:
        if shift.type == 'ems' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[3])
        if shift.type == 'observe' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[1]) 
for date in pd.date_range(start='2023-03-27', end='2023-03-31'):
    for shift in schedule.shifts:
        if shift.type == 'ems' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[4])
        if shift.type == 'observe' and shift.date == date.day:
            schedule.assign_shift(shift, schedule.employees[9])            


# ===============================================================#
decode_staff = {
    'BC': 0,
    'BW': 1,
    'KS': 2,
    'PT': 3,
    'PL': 4,
    'BT': 5,
    'BK': 6,
    'CC': 7,
    'KL': 8,
    'PU': 9,
    'NM': 10,
}




for shift in schedule.shifts:
    # 2023-03-01
    if shift.type == 'avd' and shift.date == 1:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BT']])

    # 2023-03-15
    if shift.type == 'avd' and shift.date == 15:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BC']])

    # 2023-03-20
    if shift.type == 'service 1' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'service 1+' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['NM']])
    if shift.type == 'morning conference' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PU']])
    if shift.type == 'service 2' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BT']])
    if shift.type == 'service 2+' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PT']])
    if shift.type == 'amd' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'avd' and shift.date == 20:
        schedule.assign_shift(shift, schedule.employees[decode_staff['NM']])

    # 2023-03-21
    if shift.type == 'service 1' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PL']])
    if shift.type == 'service 1+' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'morning conference' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['NM']])
    if shift.type == 'service 2' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BT']])
    if shift.type == 'service 2+' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BW']])
    if shift.type == 'amd' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PU']])
    if shift.type == 'avd' and shift.date == 21:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PU']])

    # # 2023-03-22
    if shift.type == 'service 1' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KS']])
    if shift.type == 'service 1+' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BK']])
    if shift.type == 'morning conference' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BW']])
    if shift.type == 'service 2' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PL']])
    if shift.type == 'service 2+' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['NM']])
    if shift.type == 'amd' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KS']])
    if shift.type == 'avd' and shift.date == 22:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BK']])

    # # 2023-03-23
    if shift.type == 'service 1' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BK']])
    if shift.type == 'service 1+' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'morning conference' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PT']])
    if shift.type == 'service 2' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PU']])
    if shift.type == 'service 2+' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PL']])
    if shift.type == 'amd' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'avd' and shift.date == 23:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KS']])
    
    # # 2023-03-24
    if shift.type == 'service 1' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['BK']])
    if shift.type == 'service 1+' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['CC']])
    if shift.type == 'morning conference' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'service 2' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['PT']])
    if shift.type == 'service 2+' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KL']])
    if shift.type == 'amd' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['CC']])
    if shift.type == 'avd' and shift.date == 24:
        schedule.assign_shift(shift, schedule.employees[decode_staff['KS']])

    

# ===============================================================#

def add_not_available(employee, morning_task_date, afternoon_task_date, all_day_task_date):
    for date in morning_task_date:
        morning_task = Task(name='morning_task', description="", start_time=datetime(2023, 3, date, 8, 0), duration=timedelta(hours=4))
        employee.add_task(morning_task)
    for date in afternoon_task_date:
        afternoon_task = Task(name='afternoon_task', description="", start_time=datetime(2023, 3, date, 12, 0), duration=timedelta(hours=4))
        employee.add_task(afternoon_task)
    for date in all_day_task_date:
        all_day_task = Task(name='all_day_task', description="", start_time=datetime(2023, 3, date, 8, 0), duration=timedelta(hours=8))
        employee.add_task(all_day_task)



# Add employee's tasks
# บริบูรณ์ เชนธนากิจ (BC)
morning_task_date = [7, 8, 9, 13, 14, 20, 21, 22, 23, 24, 27, 28, ]
all_day_task_date = []
afternoon_task_date = [7, 10, 20, 22, 23, 24, 29, 31, ]
add_not_available(schedule.employees[0], morning_task_date, afternoon_task_date, all_day_task_date)

# บวร วิทยชำนาญกุล (BW)
morning_task_date = [1, 3, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 23, 28, 29, 30, 31, ]
all_day_task_date = []
afternoon_task_date = [1, 2, 10, 11, 12, 13, 16, 17, 20, 23, 24, 28, 29, 30, 31, ]
add_not_available(schedule.employees[1], morning_task_date, afternoon_task_date, all_day_task_date)

# กรองกาญจน์ สุธรรม (KS)
morning_task_date = [1, 2, 10, 15, 16, 17, 18, 19, 20, 21, 25, 26, ]
all_day_task_date = []
afternoon_task_date = [1, 2, 3, 9, 10, 13, 20, 21, 25, 26, ]
add_not_available(schedule.employees[2], morning_task_date, afternoon_task_date, all_day_task_date)

# ปริญญา เทียนวิบูลย์ (PT)
morning_task_date = [1, 2, 3, 4, 5, 6, 7, 9, 13, 14, 15, 20, 21, 22, 24, 27, ]
all_day_task_date = []
afternoon_task_date = [1, 2, 3, 4, 5, 6, 7, 15, 16, 22, 23, ]
add_not_available(schedule.employees[3], morning_task_date, afternoon_task_date, all_day_task_date)

# ภาวิตา เลาหกุล (PL)
morning_task_date = [1, 2, 3, 4, 5, 6, 7, 10, 20, 22, ]
all_day_task_date = []
afternoon_task_date = [1, 2, 3, 4, 6, 10, 13, 15, 20, 21, 31, ]
add_not_available(schedule.employees[4], morning_task_date, afternoon_task_date, all_day_task_date)

# ธีรพล ตั้งสุวรรณรักษ์ (BT)
morning_task_date = [10, 15, 16, 20, 21, 22, 23, 24, ]
all_day_task_date = []
afternoon_task_date = [3, 8, 10, 13, 17, 22, 23, 24, 27, 28, 29, ]
add_not_available(schedule.employees[5], morning_task_date, afternoon_task_date, all_day_task_date)

# บุญฤทธิ์ คำทิพย์ (BK)
morning_task_date = [1, 2, 3, 7, 8, 9, 10, 13, 15, 18, 19, 20, 21, 25, 26, 27, ]
all_day_task_date = []
afternoon_task_date = [1, 2, 3, 7, 8, 9, 10, 15, 20, 21, 23, 24, ]
add_not_available(schedule.employees[6], morning_task_date, afternoon_task_date, all_day_task_date)

# ชานนท์ ช่างรัตนากร (CC)
morning_task_date = [9, 13, 15, 16, 17, 18, 19, 20, 22, 23, 25, 26, 27, 28, ]
all_day_task_date = []
afternoon_task_date = [7, 8, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, ]
add_not_available(schedule.employees[7], morning_task_date, afternoon_task_date, all_day_task_date)

# กอสิน เลาหะวิสุทธิ์ (KL)
morning_task_date = [1, 8, 15, 22, 29, ]
all_day_task_date = []
afternoon_task_date = [1, 8, 15, 21, 22, 25, 26, 27, 28, 29, ]
add_not_available(schedule.employees[8], morning_task_date, afternoon_task_date, all_day_task_date)

# พิมพ์พรรณ อัศวสุรอิน (PU)
morning_task_date = [1, 2, 3, 8, 10, 15, 17, 22, 24, 29, 31, ]
all_day_task_date = []
afternoon_task_date = [2, 8, 15, 22, 24, 29, ]
add_not_available(schedule.employees[9], morning_task_date, afternoon_task_date, all_day_task_date)

# ณัฐฐิกานต์ มีลาภ (NM)
morning_task_date = [1, 3, 7, 8, 9, 10, 15, 17, 19, 22, 23, 24, 27, 29, 31, ]
all_day_task_date = []
afternoon_task_date = [2, 7, 8, 9, 10, 15, 19, 21, 23, 24, 29, ]
add_not_available(schedule.employees[10], morning_task_date, afternoon_task_date, all_day_task_date)

# Add dummy employee
# schedule.add_employee(Employee(first_name='Unassigned', last_name= '-', role = 'unassigned'))

# # # solve the schedule
schedule.solve()
schedule.to_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/outputs')



Solution 32
  Objective value = 2147


,morning conference,ems,avd,service 2+,service 1,amd,observe,service 2,service 1+
2023-03-01,[ธีรพล],[บริบูรณ์],[ธีรพล],[พิมพ์พรรณ],[ชานนท์],,[บุญฤทธิ์],[ณัฐฐิกานต์],[บริบูรณ์]
2023-03-02,[ณัฐฐิกานต์],[บริบูรณ์],[ชานนท์],[ชานนท์],[ธีรพล],,[บุญฤทธิ์],[กอสิน],[บวร]
2023-03-03,[ชานนท์],[บริบูรณ์],[ชานนท์],[พิมพ์พรรณ],[กรองกาญจน์],,[บุญฤทธิ์],[บวร],[ธีรพล]
2023-03-04,,,[บุญฤทธิ์],,,,,,
2023-03-05,,,[กรองกาญจน์],,,,,,
2023-03-06,,,[บวร],,,,,,
2023-03-07,[กอสิน],[ธีรพล],[บวร],[ภาวิตา],[พิมพ์พรรณ],,[กรองกาญจน์],[กอสิน],[ชานนท์]
2023-03-08,[ภาวิตา],[ธีรพล],[ปริญญา],[บริบูรณ์],[บวร],,[กรองกาญจน์],[ภาวิตา],[ชานนท์]
2023-03-09,,[ธีรพล],[กอสิน],[ภาวิตา],[บวร],,[กรองกาญจน์],[ปริญญา],[พิมพ์พรรณ]
2023-03-10,,[ธีรพล],[กอสิน],[ชานนท์],[ปริญญา],,[กรองกาญจน์],[พิมพ์พรรณ],[กอสิน]


,morning conference,ems,avd,service 2+,service 1,amd,observe,service 2,service 1+
บริบูรณ์ เชนธนากิจ,1,3,3,2,2,1,0,2,3
บวร วิทยชำนาญกุล,1,0,3,2,2,1,5,2,1
กรองกาญจน์ สุธรรม,1,0,3,2,2,2,4,2,3
ปริญญา เทียนวิบูลย์,2,5,3,1,2,1,0,2,1
ภาวิตา เลาหกุล,1,5,0,3,2,1,5,2,2
ธีรพล ตั้งสุวรรณรักษ์,2,4,3,2,2,1,0,2,3
บุญฤทธิ์ คำทิพย์,2,0,3,2,2,2,3,2,1
ชานนท์ ช่างรัตนากร,2,0,3,2,2,2,0,2,3
กอสิน เลาหะวิสุทธิ์,2,0,4,2,2,2,0,2,3
พิมพ์พรรณ อัศวสุรอิน,1,0,3,2,2,2,5,2,1


^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


Solution found.


,morning conference,ems,avd,service 2+,service 1,amd,observe,service 2,service 1+
2023-03-01,ธีรพล,บริบูรณ์,ธีรพล,พิมพ์พรรณ,ชานนท์,,บุญฤทธิ์,ณัฐฐิกานต์,บริบูรณ์
2023-03-02,ณัฐฐิกานต์,บริบูรณ์,ชานนท์,ชานนท์,ธีรพล,,บุญฤทธิ์,กอสิน,บวร
2023-03-03,ชานนท์,บริบูรณ์,ชานนท์,ณัฐฐิกานต์,ธีรพล,,บุญฤทธิ์,บวร,กอสิน
2023-03-04,,,บุญฤทธิ์,,,,,,
2023-03-05,,,กรองกาญจน์,,,,,,
2023-03-06,,,บวร,,,,,,
2023-03-07,กอสิน,ธีรพล,บวร,ภาวิตา,พิมพ์พรรณ,,กรองกาญจน์,กอสิน,ชานนท์
2023-03-08,ภาวิตา,ธีรพล,ปริญญา,บริบูรณ์,บวร,,กรองกาญจน์,ภาวิตา,ชานนท์
2023-03-09,,ธีรพล,กอสิน,ชานนท์,บวร,,กรองกาญจน์,ปริญญา,พิมพ์พรรณ
2023-03-10,,ธีรพล,กอสิน,พิมพ์พรรณ,ปริญญา,,กรองกาญจน์,ชานนท์,บริบูรณ์


,morning conference,ems,avd,service 2+,service 1,amd,observe,service 2,service 1+
บริบูรณ์ เชนธนากิจ,1,3,3,2,2,1,0,2,3
บวร วิทยชำนาญกุล,1,0,3,2,2,1,5,2,1
กรองกาญจน์ สุธรรม,1,0,3,2,2,2,4,2,3
ปริญญา เทียนวิบูลย์,2,5,3,1,2,1,0,2,1
ภาวิตา เลาหกุล,1,5,0,3,2,1,5,2,2
ธีรพล ตั้งสุวรรณรักษ์,2,4,3,2,2,1,0,2,3
บุญฤทธิ์ คำทิพย์,2,0,3,2,2,2,3,2,1
ชานนท์ ช่างรัตนากร,2,0,3,2,2,2,0,2,3
กอสิน เลาหะวิสุทธิ์,2,0,4,2,2,2,0,2,3
พิมพ์พรรณ อัศวสุรอิน,1,0,3,2,2,2,5,2,1


In [163]:
name_dict = {
    'BC': 'บริบูรณ์',
    'BW': 'บวร',
    'KS': 'กรองกาญจน์',
    'PT': 'ปริญญา',
    'PL': 'ภาวิตา',
    'BT': 'ธีรพล',
    'BK': 'บุญฤทธิ์',
    'CC': 'ชานนท์',
    'KL': 'กอสิน',
    'PU': 'พิมพ์พรรณ',
    'NM': 'ณัฐฐิกานต์'
}

#Reverse
name_dict = {v: k for k, v in name_dict.items()}

# Change schedule dataframe to short name of employee (with name_dict)
schedule_df = pd.read_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/outputs/schedule.csv')
schedule_df = schedule_df.applymap(lambda x: name_dict[x] if x in name_dict else x)
display(schedule_df)

schedule_df.to_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/outputs/schedule_shortname.csv', index=False)




,Unnamed: 0,morning conference,ems,avd,service 2+,service 1,amd,observe,service 2,service 1+
0,2023-03-01,BT,BC,BT,PU,CC,,BK,NM,BC
1,2023-03-02,NM,BC,CC,CC,BT,,BK,KL,BW
2,2023-03-03,CC,BC,CC,NM,BT,,BK,BW,KL
3,2023-03-04,,,BK,,,,,,
4,2023-03-05,,,KS,,,,,,
5,2023-03-06,,,BW,,,,,,
6,2023-03-07,KL,BT,BW,PL,PU,,KS,KL,CC
7,2023-03-08,PL,BT,PT,BC,BW,,KS,PL,CC
8,2023-03-09,,BT,KL,CC,BW,,KS,PT,PU
9,2023-03-10,,BT,KL,PU,PT,,KS,CC,BC
